In [23]:
import numpy as np
import pandas as pd
import os
import re
from tqdm import tqdm
import requests
from io import StringIO
from os import listdir
from os.path import isfile, join
import datetime as dt
from datetime import datetime, timedelta

In [24]:
def listdir_nohidden(path):
    for f in os.listdir(path):
        if not f.startswith('.'):
            yield f

In [25]:
#change this to the file path of the data
FILE_PATH = "/Users/evanlipton/Dropbox (YSE)/Solar_ElectricData/Data"

In [26]:
#was originally for multiple ISOs in same notebook but broke out each iso to its own notebook given how different they ended up being

ISOS= ["CAISO"]
ISO = "CAISO"

In [27]:
ISO_files = {}
ISO_data = {}
ISO_final = {}

#load in
for ISO in ISOS:
    ISO_files[ISO] = listdir_nohidden(f"{FILE_PATH}/{ISO}/old")
    ISO_data[ISO] = {}
    for file in tqdm(ISO_files[ISO]):
        compiled = re.compile(re.escape(f"{ISO}_"), re.IGNORECASE)

        ISO_data[ISO][compiled.sub("", file).replace(".csv","")] = pd.read_csv(f"{FILE_PATH}/{ISO}/old/{file}")

13it [00:01,  8.02it/s]


In [28]:
#add merge datetime
titles = {}
for key in ISO_data:
    print(key)
    titles[key]= []
    for k in ISO_data[key]:
        try:
            ISO_data[key][k]["time_tomerge"] = pd.to_datetime(ISO_data[key][k].iloc[:,0], utc=True)
        except:
            #remove if there are extra rows
            ISO_data[key][k] = ISO_data[key][k][ISO_data[key][k].index % 24 != 0]
            ISO_data[key][k]["time_tomerge"] = pd.to_datetime(ISO_data[key][k].iloc[:,0], utc=True)

CAISO


In [29]:
ISO_final = {}
def clean_old(ISO: str, df: str):

    #convert all columns to lower
    ISO_data[ISO][df].columns = ISO_data[ISO][df].columns.str.lower()

    #remove all the extra date columns
    no_date = [col for col in ISO_data[ISO][df].columns if "date" not in col]
    ISO_data[ISO][df] = ISO_data[ISO][df][no_date]

    #try to remove extra columns if they are there
    try:
        ISO_data[ISO][df] = ISO_data[ISO][df].drop("unnamed: 0")
        ISO_data[ISO][df] = ISO_data[ISO][df].drop("hour")
    except:
        pass

    #sort values by the datetime and set as the index
    ISO_data[ISO][df] = ISO_data[ISO][df].sort_values(by="time_tomerge")
    ISO_data[ISO][df] = ISO_data[ISO][df].set_index("time_tomerge")

    #save to the dict
    ISO_final[ISO] = ISO_data[ISO][df]

    #remove the iso from the column name
    for column in ISO_final[ISO].columns:
        if str(ISO).lower() in column:
            ISO_final[ISO] = ISO_final[ISO].rename(columns={column: column.replace(f"{str(ISO).lower()}_","")})
    print(ISO_final[ISO])

In [30]:
clean_old(ISO, "all_data")

                           ancillary_da_nr  ancillary_da_rd  ancillary_da_rmd  \
time_tomerge                                                                    
2014-01-01 00:00:00+00:00          40.0984         625.4000           3.90890   
2014-01-01 01:00:00+00:00          47.2970         681.4500           0.00000   
2014-01-01 02:00:00+00:00          44.8790         769.9500           0.00000   
2014-01-01 03:00:00+00:00          43.5180         769.9500           0.00000   
2014-01-01 04:00:00+00:00          42.9170         769.9500           0.00000   
...                                    ...              ...               ...   
2018-12-31 19:00:00+00:00          49.9224        3520.0000           0.00000   
2018-12-31 20:00:00+00:00          46.9029        4396.0000           0.00000   
2018-12-31 21:00:00+00:00          40.9629        4530.1095          12.21950   
2018-12-31 22:00:00+00:00          39.6770         828.7500          20.87470   
2018-12-31 23:00:00+00:00   

In [31]:
ISO_final["CAISO"].columns

Index(['ancillary_da_nr', 'ancillary_da_rd', 'ancillary_da_rmd',
       'ancillary_da_rmu', 'ancillary_da_ru', 'ancillary_da_sr',
       'ancillary_rt_nr', 'ancillary_rt_rd', 'ancillary_rt_rmd',
       'ancillary_rt_rmu', 'ancillary_rt_ru', 'ancillary_rt_sr',
       'congestion_da', 'energy_da', 'unnamed: 0', 'gas_price', 'hour',
       'gen_geothermal', 'gen_biomass', 'gen_biogas', 'gen_small_hydro',
       'gen_wind', 'gen_solar_pv', 'gen_solar_thermal', 'gen_renewables',
       'gen_nuclear', 'gen_thermal', 'gen_imports', 'gen_hydro', 'load_2da',
       'load_7da', 'load_rt', 'load_da', 'load_rt.1', 'load_rtpd',
       'losses_da'],
      dtype='object')

In [32]:
#load in the new data that was combined (files by variable)
ISO_done = ["CAISO"]
FILE_PATH = "/Users/evanlipton/Dropbox (YSE)/Solar_ElectricData/Data"
new_data = {}
for ISO in ISO_done:
    sub_folder = f"{FILE_PATH}/{ISO}/new/combined"
    files = [f for f in listdir(sub_folder) if isfile(join(sub_folder, f)) and f[0]!="."]
    new_data[ISO] = {}
    print(ISO)
    for file in tqdm(files):
        new_data[ISO][file.replace(".csv", "")] = pd.read_csv(f"{sub_folder}/{file}")

CAISO


 44%|████▍     | 4/9 [01:51<01:38, 19.79s/it]/usr/local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3442: DtypeWarning: Columns (1,2,3,5,6,7,8,9,10,11,12,18,19,20,21,26) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
100%|██████████| 9/9 [03:12<00:00, 21.37s/it]


In [70]:
def create_date(entry: tuple):
    return pd.to_datetime(pd.to_datetime(entry[0]) + timedelta(hours = int(entry[1])))

def add_caiso_ancillary(sample_data: pd.DataFrame, ANC_TYPE: str, ANC_REGION: str, RESULT_TYPE: str, add_label: str = ""):

    #subsets a specific ancillary service type
    subset = sample_data[(sample_data["ANC_TYPE"] == ANC_TYPE[13:].upper()) & (sample_data["ANC_REGION"] == ANC_REGION) & (sample_data["RESULT_TYPE"] == RESULT_TYPE)]
    print(subset)

    #removes daylight saving hour 25
    subset = subset[subset["OPR_HR"]<25]

    #groups by the hour and day and get takes the mean
    subset = pd.DataFrame(subset.groupby(['OPR_DT', "OPR_HR"])["MW"].mean())

    #creates a new column to get the date
    subset["DATE_HR"] = subset.index.to_numpy()

    #creates a date fromthat
    subset["time_tomerge"] = subset["DATE_HR"].apply(create_date)

    #clean up
    subset = subset.drop(columns = ["DATE_HR"])
    subset = subset.rename(columns={"MW": f"{ANC_TYPE}{add_label}"})
    subset = subset.drop_duplicates()

    #check length
    print(len(np.unique(subset["time_tomerge"])))
    print(len(subset["time_tomerge"]))

    return subset

In [71]:
add_caiso_ancillary(new_data["CAISO"]["ancillary_res_da"], "anc", "AS_CAISO_EXP", "AS_BUY_MW", "_mw")

,Unnamed: 0,INTERVALSTARTTIME_GMT,INTERVALENDTIME_GMT,OPR_DT,OPR_HR,OPR_INTERVAL,ANC_TYPE,ANC_REGION,GRP,MARKET_RUN_ID,RESULT_TYPE,UOM,POS,XML_DATA_ITEM,MW,GROUP,time_tomerge
58,951,2019-01-01T08:00:00-00:00,2019-01-01T09:00:00-00:00,2019-01-01,1,0,RD,AS_CAISO_EXP,Regional Detail,DAM,AS_COST,2,4,RD_TOT_CST_PRC,4587.144,7,2019-01-01 08:00:00+00:00
156,961,2019-01-01T09:00:00-00:00,2019-01-01T10:00:00-00:00,2019-01-01,2,0,RD,AS_CAISO_EXP,Regional Detail,DAM,AS_COST,2,4,RD_TOT_CST_PRC,1500.000,7,2019-01-01 09:00:00+00:00
225,957,2019-01-01T10:00:00-00:00,2019-01-01T11:00:00-00:00,2019-01-01,3,0,RD,AS_CAISO_EXP,Regional Detail,DAM,AS_COST,2,4,RD_TOT_CST_PRC,1500.000,7,2019-01-01 10:00:00+00:00
385,973,2019-01-01T11:00:00-00:00,2019-01-01T12:00:00-00:00,2019-01-01,4,0,RD,AS_CAISO_EXP,Regional Detail,DAM,AS_COST,2,4,RD_TOT_CST_PRC,1500.000,7,2019-01-01 11:00:00+00:00
450,967,2019-01-01T12:00:00-00:00,2019-01-01T13:00:00-00:00,2019-01-01,5,0,RD,AS_CAISO_EXP,Regional Detail,DAM,AS_COST,2,4,RD_TOT_CST_PRC,1500.000,7,2019-01-01 12:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9947,8437,2019-01-05T07:00:00-00:00,2019-01-05T08:00:00-00:00,2019-01-04,24,0,RD,AS_CAISO_EXP,Regional Detail,DAM,AS_COST,2,4,RD_TOT_CST_PRC,3740.607,79,2019-01-05 07:00:00+00:00
10023,10925,2019-01-05T08:00:00-00:00,2019-01-05T09:00:00-00:00,2019-01-05,1,0,RD,AS_CAISO_EXP,Regional Detail,DAM,AS_COST,2,4,RD_TOT_CST_PRC,2100.000,103,2019-01-05 08:00:00+00:00
10090,10860,2019-01-05T09:00:00-00:00,2019-01-05T10:00:00-00:00,2019-01-05,2,0,RD,AS_CAISO_EXP,Regional Detail,DAM,AS_COST,2,4,RD_TOT_CST_PRC,2103.000,103,2019-01-05 09:00:00+00:00
10203,10858,2019-01-05T10:00:00-00:00,2019-01-05T11:00:00-00:00,2019-01-05,3,0,RD,AS_CAISO_EXP,Regional Detail,DAM,AS_COST,2,4,RD_TOT_CST_PRC,2100.000,103,2019-01-05 10:00:00+00:00


In [34]:
#get the ancillary services in the old dataset
ancillaries = [col for col in ISO_final["CAISO"].columns if "ancillary" in col]
print(ancillaries)

['ancillary_da_nr', 'ancillary_da_rd', 'ancillary_da_rmd', 'ancillary_da_rmu', 'ancillary_da_ru', 'ancillary_da_sr', 'ancillary_rt_nr', 'ancillary_rt_rd', 'ancillary_rt_rmd', 'ancillary_rt_rmu', 'ancillary_rt_ru', 'ancillary_rt_sr']


In [38]:
#Calls the function to create the ancillaries
new_data["CAISO"]["total"] = pd.DataFrame({"time_tomerge": pd.date_range(start="1/1/2019", end="1/1/2023", freq="H")})
for anc in tqdm(ancillaries):
    if anc[10:12] == "da":
        new_data["CAISO"]["total"] = new_data["CAISO"]["total"].merge(add_caiso_ancillary(new_data["CAISO"]["ancillary_res_da"], anc, "AS_CAISO_EXP", "AS_COST"), on = "time_tomerge", how = "left")
    else:
        new_data["CAISO"]["total"] = new_data["CAISO"]["total"].merge(add_caiso_ancillary(new_data["CAISO"]["ancillary_res_rt"], anc, "AS_CAISO_EXP", "AS_COST"), on = "time_tomerge", how = "left")
#print(sample)

  8%|▊         | 1/12 [00:11<02:11, 11.92s/it]

35060
35060


 17%|█▋        | 2/12 [00:16<01:17,  7.78s/it]

35060
35060


 25%|██▌       | 3/12 [00:20<00:54,  6.01s/it]

...

34316
34316


 92%|█████████▏| 11/12 [01:16<00:07,  7.09s/it]

34316
34316


100%|██████████| 12/12 [01:22<00:00,  6.88s/it]

34316
34316


In [39]:
#new_data["CAISO"]["total"] = pd.DataFrame({"time_tomerge": pd.date_range(start="1/1/2019", end="1/1/2023", freq="H")})
for anc in tqdm(ancillaries):
    if anc[10:12] == "da":
        new_data["CAISO"]["total"] = new_data["CAISO"]["total"].merge(add_caiso_ancillary(new_data["CAISO"]["ancillary_res_da"], anc, "AS_CAISO_EXP", "AS_BUY_MW", "_mw"), on = "time_tomerge", how = "left")
    else:
        new_data["CAISO"]["total"] = new_data["CAISO"]["total"].merge(add_caiso_ancillary(new_data["CAISO"]["ancillary_res_rt"], anc, "AS_CAISO_EXP", "AS_BUY_MW", "_mw"), on = "time_tomerge", how = "left")

  8%|▊         | 1/12 [00:04<00:52,  4.78s/it]

35060
35060


 17%|█▋        | 2/12 [00:08<00:41,  4.11s/it]

35060
35060


 25%|██▌       | 3/12 [00:12<00:35,  3.93s/it]

...

34316
34316


 92%|█████████▏| 11/12 [00:55<00:05,  5.91s/it]

34316
34316


100%|██████████| 12/12 [01:00<00:00,  5.07s/it]

34316
34316


In [40]:
print(new_data["CAISO"]["total"])

             time_tomerge  ancillary_da_nr  ancillary_da_rd  ancillary_da_rmd  \
0     2019-01-01 00:00:00              NaN              NaN               NaN   
1     2019-01-01 01:00:00          73.6890         4587.144               0.0   
2     2019-01-01 02:00:00          70.5892         1500.000               0.0   
3     2019-01-01 03:00:00          71.0776         1500.000               0.0   
4     2019-01-01 04:00:00          73.1093         1500.000               0.0   
...                   ...              ...              ...               ...   
35060 2022-12-31 20:00:00         101.5209         5700.000               0.0   
35061 2022-12-31 21:00:00          90.5520         6612.000               0.0   
35062 2022-12-31 22:00:00          87.7104         8850.000               0.0   
35063 2022-12-31 23:00:00          84.4800         5670.000               0.0   
35064 2023-01-01 00:00:00          80.8572         7650.000               0.0   

       ancillary_da_rmu  an

In [41]:
ISO_final["CAISO"].columns

Index(['ancillary_da_nr', 'ancillary_da_rd', 'ancillary_da_rmd',
       'ancillary_da_rmu', 'ancillary_da_ru', 'ancillary_da_sr',
       'ancillary_rt_nr', 'ancillary_rt_rd', 'ancillary_rt_rmd',
       'ancillary_rt_rmu', 'ancillary_rt_ru', 'ancillary_rt_sr',
       'congestion_da', 'energy_da', 'unnamed: 0', 'gas_price', 'hour',
       'gen_geothermal', 'gen_biomass', 'gen_biogas', 'gen_small_hydro',
       'gen_wind', 'gen_solar_pv', 'gen_solar_thermal', 'gen_renewables',
       'gen_nuclear', 'gen_thermal', 'gen_imports', 'gen_hydro', 'load_2da',
       'load_7da', 'load_rt', 'load_da', 'load_rt.1', 'load_rtpd',
       'losses_da'],
      dtype='object')

In [42]:
def add_caiso_energy(sample_data: pd.DataFrame, LMP_TYPE: str, col_name: str):

    #get the lmp type and removes the daylight saving hour
    subset = sample_data[(sample_data["LMP_TYPE"] == LMP_TYPE)]
    subset = subset[subset["OPR_HR"]<25]

    #takes the mean for the hour and day of each the lmp type
    try:
        subset = pd.DataFrame(subset.groupby(['OPR_DT', "OPR_HR"])["MW"].mean())
    except:
        subset = pd.DataFrame(subset.groupby(['OPR_DT', "OPR_HR"])["VALUE"].mean())

    #creates a date
    subset["DATE_HR"] = subset.index.to_numpy()
    subset["time_tomerge"] = subset["DATE_HR"].apply(create_date)
    #subset["time_tomerge"] = subset["time_tomerge"].map(lambda x: x.tz_convert('US/Pacific'))

    #clean up
    subset = subset.drop(columns = ["DATE_HR"])
    subset = subset.reset_index(drop=True)


    subset = subset.rename(columns={"MW": col_name})
    subset = subset.rename(columns={"VALUE": col_name})
    print(len(subset))
    return subset

In [43]:
new_data["CAISO"]["total"] = new_data["CAISO"]["total"].merge(add_caiso_energy(new_data["CAISO"]["energy_da"], "MCC", "congestion_da"), on = "time_tomerge", how="left")
new_data["CAISO"]["total"] = new_data["CAISO"]["total"].merge(add_caiso_energy(new_data["CAISO"]["energy_da"], "MCE", "energy_da"), on = "time_tomerge", how="left")
new_data["CAISO"]["total"] = new_data["CAISO"]["total"].merge(add_caiso_energy(new_data["CAISO"]["energy_da"], "MCL", "losses_da"), on = "time_tomerge",how="left")

print(new_data["CAISO"]["total"])

27957
27957
27957
             time_tomerge  ancillary_da_nr  ancillary_da_rd  ancillary_da_rmd  \
0     2019-01-01 00:00:00              NaN              NaN               NaN   
1     2019-01-01 01:00:00          73.6890         4587.144               0.0   
2     2019-01-01 02:00:00          70.5892         1500.000               0.0   
3     2019-01-01 03:00:00          71.0776         1500.000               0.0   
4     2019-01-01 04:00:00          73.1093         1500.000               0.0   
...                   ...              ...              ...               ...   
35060 2022-12-31 20:00:00         101.5209         5700.000               0.0   
35061 2022-12-31 21:00:00          90.5520         6612.000               0.0   
35062 2022-12-31 22:00:00          87.7104         8850.000               0.0   
35063 2022-12-31 23:00:00          84.4800         5670.000               0.0   
35064 2023-01-01 00:00:00          80.8572         7650.000               0.0   

       an

In [44]:
print(np.unique(new_data["CAISO"]["energy_da"]["NODE"]))

['TH_NP15_GEN-APND' 'TH_SP15_GEN-APND' 'TH_ZP26_GEN-APND']


In [45]:
def add_caiso_gas(sample_data: pd.DataFrame, col_name: str):
    subset = sample_data
    subset = subset[subset["OPR_HR"]<25]

    subset = pd.DataFrame(subset.groupby(['OPR_DT', "OPR_HR"])["PRC"].mean())
    subset["DATE_HR"] = subset.index.to_numpy()
    subset["time_tomerge"] = subset["DATE_HR"].apply(create_date)
    subset = subset.drop(columns = ["DATE_HR"])
    subset = subset.reset_index(drop=True)
    subset = subset.rename(columns={"PRC": col_name})
    print(len(subset))
    return subset

In [46]:
new_data["CAISO"]["total"] = new_data["CAISO"]["total"].merge(add_caiso_gas(new_data["CAISO"]["gas_price"], "gas_price"), on = "time_tomerge", how="left")
new_data["CAISO"]["total"]

35036


,time_tomerge,ancillary_da_nr,ancillary_da_rd,ancillary_da_rmd,ancillary_da_rmu,ancillary_da_ru,ancillary_da_sr,ancillary_rt_nr,ancillary_rt_rd,ancillary_rt_rmd,...,ancillary_rt_nr_mw,ancillary_rt_rd_mw,ancillary_rt_rmd_mw,ancillary_rt_rmu_mw,ancillary_rt_ru_mw,ancillary_rt_sr_mw,congestion_da,energy_da,losses_da,gas_price
0,2019-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-01-01 01:00:00,73.6890,4587.144,0.0,0.0,2451.00000,669.90000,0.0,0.734587,0.0,...,0.0,0.3825,300.1300,5235.9000,0.3525,21.0650,NaN,NaN,NaN,5.098778
2,2019-01-01 02:00:00,70.5892,1500.000,0.0,0.0,1797.00000,641.72000,0.0,0.202735,0.0,...,0.0,0.4075,300.0000,2788.1600,0.4400,0.0000,NaN,NaN,NaN,5.098778
3,2019-01-01 03:00:00,71.0776,1500.000,0.0,0.0,1512.00000,161.54000,0.0,0.451492,0.0,...,0.0,0.3600,300.0000,2822.8600,0.3525,47.3900,NaN,NaN,NaN,5.098778
4,2019-01-01 04:00:00,73.1093,1500.000,0.0,0.0,1512.00000,166.15750,0.0,0.437500,0.0,...,0.0,0.3500,310.2375,757.5000,0.3300,47.4100,NaN,NaN,NaN,5.098778
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35060,2022-12-31 20:00:00,101.5209,5700.000,0.0,0.0,2483.99900,2533.73540,0.0,106.494525,0.0,...,0.0,38.6550,579.0100,740.0350,4.0600,3.7500,-0.030910,140.78702,-3.402353,16.604248
35061,2022-12-31 21:00:00,90.5520,6612.000,0.0,0.0,1892.67000,1916.61000,0.0,214.631892,0.0,...,0.0,77.3100,1785.3200,641.2800,8.1225,15.6250,-0.004903,135.76392,-3.267383,16.604248
35062,2022-12-31 22:00:00,87.7104,8850.000,0.0,0.0,4625.81000,3186.47000,0.0,0.000000,0.0,...,0.0,81.6000,739.1500,849.9500,8.1250,18.2200,0.000000,128.26624,-3.031357,16.604248
35063,2022-12-31 23:00:00,84.4800,5670.000,0.0,0.0,1040.81446,845.73324,0.0,162.101250,0.0,...,0.0,72.0450,634.1400,842.6300,15.7875,102.6125,0.000000,121.29756,-3.141607,16.604248


In [48]:
#got the 2022 energy rt data from a different source so formatting is different
new_data["CAISO"]["energy_rt1"] = new_data["CAISO"]["energy_rt"][['INTERVALSTARTTIME_GMT', 'INTERVALENDTIME_GMT', 'OPR_DT',
       'OPR_HR', 'NODE_ID_XML', 'NODE_ID', 'NODE', 'MARKET_RUN_ID', 'LMP_TYPE',
       'XML_DATA_ITEM', 'PNODE_RESMRID', 'GRP_TYPE', 'POS', 'VALUE',
       'OPR_INTERVAL', 'GROUP', 'time_tomerge']][new_data["CAISO"]["energy_rt"]["INTERVALSTARTTIME_GMT"].notna()]
new_data["CAISO"]["energy_rt2"] = new_data["CAISO"]["energy_rt"][['Time', 'Market', 'Location',
       'Location Type', 'LMP', 'Energy', 'Congestion', 'Loss', 'time_tomerge']][new_data["CAISO"]["energy_rt"]["INTERVALSTARTTIME_GMT"].isna()]
new_data["CAISO"]["energy_rt2"]["time_tomerge"] = pd.to_datetime(new_data["CAISO"]["energy_rt2"]["Time"],utc =True)
print(new_data["CAISO"]["energy_rt1"].tail())
print(new_data["CAISO"]["energy_rt2"].head())

             INTERVALSTARTTIME_GMT        INTERVALENDTIME_GMT      OPR_DT  \
3378199  2022-01-01T07:55:00-00:00  2022-01-01T08:00:00-00:00  2021-12-31   
3378200  2022-01-01T07:55:00-00:00  2022-01-01T08:00:00-00:00  2021-12-31   
3378201  2022-01-01T07:55:00-00:00  2022-01-01T08:00:00-00:00  2021-12-31   
3378202  2022-01-01T07:55:00-00:00  2022-01-01T08:00:00-00:00  2021-12-31   
3378203  2022-01-01T07:55:00-00:00  2022-01-01T08:00:00-00:00  2021-12-31   

         OPR_HR       NODE_ID_XML           NODE_ID              NODE  \
3378199    24.0  TH_SP15_GEN-APND  TH_SP15_GEN-APND  TH_SP15_GEN-APND   
3378200    24.0  TH_NP15_GEN-APND  TH_NP15_GEN-APND  TH_NP15_GEN-APND   
3378201    24.0  TH_NP15_GEN-APND  TH_NP15_GEN-APND  TH_NP15_GEN-APND   
3378202    24.0  TH_NP15_GEN-APND  TH_NP15_GEN-APND  TH_NP15_GEN-APND   
3378203    24.0  TH_SP15_GEN-APND  TH_SP15_GEN-APND  TH_SP15_GEN-APND   

        MARKET_RUN_ID LMP_TYPE XML_DATA_ITEM     PNODE_RESMRID     GRP_TYPE  \
3378199           R

In [49]:
print((new_data["CAISO"]["energy_rt1"]["VALUE"][new_data["CAISO"]["energy_rt1"]["LMP_TYPE"]=="MCE"].head(36)))

4      25.13716
5      25.13716
11     25.13716
16     29.68177
20     29.68177
21     29.68177
30     30.95285
33     30.95285
34     30.95285
48     30.20531
52     30.20531
58     30.20531
61     34.22214
67     34.22214
72     34.22214
77     35.16375
79     35.16375
88     35.16375
92     33.14584
96     33.14584
103    33.14584
105    35.13061
115    35.13061
118    35.13061
122    35.13061
125    35.13061
127    35.13061
135    34.18862
136    34.18862
149    34.18862
150    31.87946
151    31.87946
160    31.87946
167    31.32310
173    31.32310
178    31.32310
Name: VALUE, dtype: float64


In [50]:
new_data["CAISO"]["total"] = new_data["CAISO"]["total"].merge(add_caiso_energy(new_data["CAISO"]["energy_rt1"], "MCC", "congestion_rt"), on = "time_tomerge",how="left")
new_data["CAISO"]["total"] = new_data["CAISO"]["total"].merge(add_caiso_energy(new_data["CAISO"]["energy_rt1"], "MCE", "energy_rt"), on = "time_tomerge",how="left")
new_data["CAISO"]["total"] = new_data["CAISO"]["total"].merge(add_caiso_energy(new_data["CAISO"]["energy_rt1"], "MCL", "losses_rt"), on = "time_tomerge", how="left")
print(new_data["CAISO"]["total"])

18766
18766
18766
             time_tomerge  ancillary_da_nr  ancillary_da_rd  ancillary_da_rmd  \
0     2019-01-01 00:00:00              NaN              NaN               NaN   
1     2019-01-01 01:00:00          73.6890         4587.144               0.0   
2     2019-01-01 02:00:00          70.5892         1500.000               0.0   
3     2019-01-01 03:00:00          71.0776         1500.000               0.0   
4     2019-01-01 04:00:00          73.1093         1500.000               0.0   
...                   ...              ...              ...               ...   
35060 2022-12-31 20:00:00         101.5209         5700.000               0.0   
35061 2022-12-31 21:00:00          90.5520         6612.000               0.0   
35062 2022-12-31 22:00:00          87.7104         8850.000               0.0   
35063 2022-12-31 23:00:00          84.4800         5670.000               0.0   
35064 2023-01-01 00:00:00          80.8572         7650.000               0.0   

       an

In [51]:
new_data["CAISO"]["total"]["time_tomerge"] = pd.to_datetime(new_data["CAISO"]["total"]["time_tomerge"], utc=True)
new_data["CAISO"]["total"] = new_data["CAISO"]["total"].set_index("time_tomerge")

In [52]:
subset = new_data["CAISO"]["energy_rt2"]
subset["time_tomerge"] = pd.to_datetime(subset["time_tomerge"], utc = True).dt.floor("H")
subset = pd.DataFrame(subset.groupby(["time_tomerge"])["Energy", "Congestion", "Loss"].mean())
subset = subset.drop_duplicates().rename(columns={'Energy':'energy_rt','Congestion':'congestion_rt', "Loss": "losses_rt"})
#print(subset)
new_data["CAISO"]["total"].update(subset)
print(new_data["CAISO"]["total"])

                           ancillary_da_nr  ancillary_da_rd  ancillary_da_rmd  \
time_tomerge                                                                    
2019-01-01 00:00:00+00:00              NaN              NaN               NaN   
2019-01-01 01:00:00+00:00          73.6890         4587.144               0.0   
2019-01-01 02:00:00+00:00          70.5892         1500.000               0.0   
2019-01-01 03:00:00+00:00          71.0776         1500.000               0.0   
2019-01-01 04:00:00+00:00          73.1093         1500.000               0.0   
...                                    ...              ...               ...   
2022-12-31 20:00:00+00:00         101.5209         5700.000               0.0   
2022-12-31 21:00:00+00:00          90.5520         6612.000               0.0   
2022-12-31 22:00:00+00:00          87.7104         8850.000               0.0   
2022-12-31 23:00:00+00:00          84.4800         5670.000               0.0   
2023-01-01 00:00:00+00:00   

/var/folders/cz/nv3pr_214md_31815rhfvdkh0000gn/T/ipykernel_48675/2296779494.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  subset = pd.DataFrame(subset.groupby(["time_tomerge"])["Energy", "Congestion", "Loss"].mean())


In [53]:
load_types = ((new_data["CAISO"]["load"].columns))
#print(np.sum(new_data["CAISO"]["load"]["TAC_AREA_NAME"]==))
print(load_types)

Index(['Unnamed: 0', 'INTERVALSTARTTIME_GMT', 'INTERVALENDTIME_GMT',
       'LOAD_TYPE', 'OPR_DT', 'OPR_HR', 'OPR_INTERVAL', 'MARKET_RUN_ID',
       'TAC_AREA_NAME', 'LABEL', 'XML_DATA_ITEM', 'POS', 'MW',
       'EXECUTION_TYPE', 'GROUP', 'time_tomerge'],
      dtype='object')


In [54]:
def add_caiso_load(sample_data: pd.DataFrame, LOAD_TYPE: str, col_name: str):
    subset = sample_data[(sample_data["EXECUTION_TYPE"] == LOAD_TYPE)]
    subset = subset[subset["OPR_HR"]<25]

    subset = pd.DataFrame(subset.groupby(['OPR_DT', "OPR_HR"])["MW"].mean())
    subset["DATE_HR"] = subset.index.to_numpy()
    subset["time_tomerge"] = subset["DATE_HR"].apply(create_date)
    subset["time_tomerge"] = pd.to_datetime(subset["time_tomerge"], utc = True)

    #subset["time_tomerge"] = subset["time_tomerge"].map(lambda x: x.tz_convert('US/Pacific'))

    subset = subset.drop(columns = ["DATE_HR"])
    subset = subset.reset_index(drop=True)


    subset = subset.rename(columns={"MW": col_name})
    print(len(subset))
    return subset

In [55]:
new_data["CAISO"]["total"] = new_data["CAISO"]["total"].merge(add_caiso_load(new_data["CAISO"]["load"][new_data["CAISO"]["load"]["TAC_AREA_NAME"] == "CA ISO-TAC"], "2DA", "load_2da"), on = "time_tomerge", how="left")
new_data["CAISO"]["total"] = new_data["CAISO"]["total"].merge(add_caiso_load(new_data["CAISO"]["load"][new_data["CAISO"]["load"]["TAC_AREA_NAME"] == "CA ISO-TAC"], "7DA", "load_7da"), on = "time_tomerge", how="left")
new_data["CAISO"]["total"] = new_data["CAISO"]["total"].merge(add_caiso_load(new_data["CAISO"]["load"][new_data["CAISO"]["load"]["TAC_AREA_NAME"] == "CA ISO-TAC"], "ACTUAL", "load_rt"), on = "time_tomerge",how="left")
new_data["CAISO"]["total"] = new_data["CAISO"]["total"].merge(add_caiso_load(new_data["CAISO"]["load"][new_data["CAISO"]["load"]["TAC_AREA_NAME"] == "CA ISO-TAC"], "DAM", "load_da"), on = "time_tomerge",how="left")
new_data["CAISO"]["total"] = new_data["CAISO"]["total"].merge(add_caiso_load(new_data["CAISO"]["load"][new_data["CAISO"]["load"]["TAC_AREA_NAME"] == "CA ISO-TAC"], "RTPD", "load_rtpd"), on = "time_tomerge",how="left")
new_data["CAISO"]["total"] = new_data["CAISO"]["total"].merge(add_caiso_load(new_data["CAISO"]["load"][new_data["CAISO"]["load"]["TAC_AREA_NAME"] == "CA ISO-TAC"], "RTD", "load_rt.1"), on = "time_tomerge",how="left")

print(new_data["CAISO"]["total"])

35012
35012
35012
35012
34990
35006
                   time_tomerge  ancillary_da_nr  ancillary_da_rd  \
0     2019-01-01 00:00:00+00:00              NaN              NaN   
1     2019-01-01 01:00:00+00:00          73.6890         4587.144   
2     2019-01-01 02:00:00+00:00          70.5892         1500.000   
3     2019-01-01 03:00:00+00:00          71.0776         1500.000   
4     2019-01-01 04:00:00+00:00          73.1093         1500.000   
...                         ...              ...              ...   
35060 2022-12-31 20:00:00+00:00         101.5209         5700.000   
35061 2022-12-31 21:00:00+00:00          90.5520         6612.000   
35062 2022-12-31 22:00:00+00:00          87.7104         8850.000   
35063 2022-12-31 23:00:00+00:00          84.4800         5670.000   
35064 2023-01-01 00:00:00+00:00          80.8572         7650.000   

       ancillary_da_rmd  ancillary_da_rmu  ancillary_da_ru  ancillary_da_sr  \
0                   NaN               NaN              N

In [56]:
#new_data["CAISO"]["generation"] = new_data["CAISO"]["generation"].drop(columns=["Unnamed: 0", "Unnamed: 0.1"])
new_data["CAISO"]["generation"]["time_tomerge"] = pd.to_datetime(new_data["CAISO"]["generation"]["Time"], utc=True)
#new_data["CAISO"]["generation"]["time_tomerge"] = new_data["CAISO"]["generation"]["time_tomerge"] + timedelta(hours=7)
new_data["CAISO"]["generation"]

,Unnamed: 0,Unnamed: 0.1,Time,Solar,Wind,Geothermal,Biomass,Biogas,Small Hydro,Coal,Nuclear,Natural Gas,Large Hydro,Batteries,Imports,Other,time_tomerge
0,0,0,2022-01-01 00:00:00-08:00,-32.0,3829.0,882.0,281.0,205.0,146.0,20.0,2264.0,7280.0,1200.0,-116.0,8200.0,0.0,2022-01-01 08:00:00+00:00
1,0,0,2021-01-01 00:00:00-08:00,-41.0,1424.0,958.0,309.0,199.0,144.0,13.0,1144.0,8069.0,690.0,-46.0,9541.0,0.0,2021-01-01 08:00:00+00:00
2,0,0,2020-01-01 00:00:00-08:00,-33.0,808.0,651.0,305.0,223.0,222.0,11.0,2274.0,7871.0,1561.0,0.0,8653.0,0.0,2020-01-01 08:00:00+00:00
3,0,0,2019-01-01 00:00:00-08:00,0.0,2810.0,993.0,380.0,225.0,200.0,11.0,2273.0,7326.0,1924.0,6.0,6254.0,0.0,2019-01-01 08:00:00+00:00
4,1,1,2021-01-01 00:05:00-08:00,-41.0,1449.0,959.0,309.0,201.0,143.0,13.0,1145.0,8084.0,698.0,-20.0,9388.0,0.0,2021-01-01 08:05:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419817,105380,105380,2020-12-31 23:35:00-08:00,-41.0,1356.0,954.0,308.0,200.0,144.0,12.0,1144.0,8078.0,701.0,-19.0,9830.0,0.0,2021-01-01 07:35:00+00:00
419818,105381,105381,2020-12-31 23:40:00-08:00,-41.0,1374.0,953.0,308.0,199.0,143.0,12.0,1144.0,8047.0,704.0,-31.0,9829.0,0.0,2021-01-01 07:40:00+00:00
419819,105382,105382,2020-12-31 23:45:00-08:00,-41.0,1378.0,954.0,309.0,200.0,143.0,13.0,1144.0,8067.0,696.0,-33.0,9786.0,0.0,2021-01-01 07:45:00+00:00
419820,105383,105383,2020-12-31 23:50:00-08:00,-41.0,1387.0,957.0,308.0,199.0,143.0,12.0,1145.0,8037.0,688.0,-47.0,9752.0,0.0,2021-01-01 07:50:00+00:00


In [57]:
print(ISO_final["CAISO"][[col for col in ISO_final["CAISO"] if "gen" in col]])

                           gen_geothermal  gen_biomass  gen_biogas  \
time_tomerge                                                         
2014-01-01 00:00:00+00:00           849.0        379.0       175.0   
2014-01-01 01:00:00+00:00           849.0        377.0       176.0   
2014-01-01 02:00:00+00:00           849.0        375.0       176.0   
2014-01-01 03:00:00+00:00           849.0        372.0       176.0   
2014-01-01 04:00:00+00:00           849.0        377.0       176.0   
...                                   ...          ...         ...   
2018-12-31 19:00:00+00:00           993.0        389.0       228.0   
2018-12-31 20:00:00+00:00           991.0        383.0       228.0   
2018-12-31 21:00:00+00:00           992.0        383.0       227.0   
2018-12-31 22:00:00+00:00           993.0        382.0       226.0   
2018-12-31 23:00:00+00:00           993.0        379.0       226.0   

                           gen_small_hydro  gen_wind  gen_solar_pv  \
time_tomerge       

In [58]:
new_data["CAISO"]["generation"] = new_data["CAISO"]["generation"].sort_values(by="time_tomerge")
#print(new_data["CAISO"]["generation"])

subset = new_data["CAISO"]["generation"]
subset["time_tomerge"] = pd.to_datetime(subset["time_tomerge"], utc=True).dt.floor("H")
subset = pd.DataFrame(subset.groupby(["time_tomerge"])[subset.columns[:-1]].mean())
subset["time_tomerge"] =subset.index
subset = subset.reset_index(drop=True)
subset = subset.rename(columns={"Solar": "gen_solar", "Wind": "gen_wind", "Geothermal": "gen_geothermal", "Biomass": "gen_biomass", "Biogas": "gen_biogas", "Small Hydro": "gen_small_hydro", "Coal": "gen_coal", "Nuclear": "gen_nuclear", "Natural Gas": "gen_natural_gas", "Large Hydro": "gen_hydro", "Batteries": "gen_batteries", "Imports": "gen_imports", "Other": "gen_other"})
print(subset)
print(len(subset))
new_data["CAISO"]["total"] = new_data["CAISO"]["total"].merge(subset, on = "time_tomerge",how="left")
print(new_data["CAISO"]["total"].set_index("time_tomerge"))
print(len(new_data["CAISO"]["total"]))

       Unnamed: 0  Unnamed: 0.1  gen_solar     gen_wind  gen_geothermal  \
0             5.5           5.5   0.000000  2812.083333      992.583333   
1            17.5          17.5   0.000000  2499.500000      991.833333   
2            29.5          29.5   0.000000  2387.083333      992.083333   
3            41.5          41.5   0.000000  2369.333333      993.250000   
4            53.5          53.5   0.000000  2323.083333      991.583333   
...           ...           ...        ...          ...             ...   
34983    105037.5      105037.5 -27.916667  3714.500000      901.333333   
34984    105049.5      105049.5 -28.500000  4078.750000      900.333333   
34985    105061.5      105061.5 -36.166667  3580.666667      900.916667   
34986    105073.5      105073.5 -37.166667  3161.166667      900.416667   
34987    105085.5      105085.5 -37.166667  2914.750000      902.250000   

       gen_biomass  gen_biogas  gen_small_hydro   gen_coal  gen_nuclear  \
0       378.500000  220.

In [59]:
new_data["CAISO"]["total"] = new_data["CAISO"]["total"].set_index("time_tomerge")
print(new_data["CAISO"]["total"])
print(ISO_final["CAISO"])

                           ancillary_da_nr  ancillary_da_rd  ancillary_da_rmd  \
time_tomerge                                                                    
2019-01-01 00:00:00+00:00              NaN              NaN               NaN   
2019-01-01 01:00:00+00:00          73.6890         4587.144               0.0   
2019-01-01 02:00:00+00:00          70.5892         1500.000               0.0   
2019-01-01 03:00:00+00:00          71.0776         1500.000               0.0   
2019-01-01 04:00:00+00:00          73.1093         1500.000               0.0   
...                                    ...              ...               ...   
2022-12-31 20:00:00+00:00         101.5209         5700.000               0.0   
2022-12-31 21:00:00+00:00          90.5520         6612.000               0.0   
2022-12-31 22:00:00+00:00          87.7104         8850.000               0.0   
2022-12-31 23:00:00+00:00          84.4800         5670.000               0.0   
2023-01-01 00:00:00+00:00   

In [61]:
final = pd.concat([ISO_final["CAISO"], new_data["CAISO"]["total"]])
final.to_csv("/Users/evanlipton/Dropbox (YSE)/Solar_ElectricData/ISOs/CAISO.csv")

In [63]:
final.columns

Index(['ancillary_da_nr', 'ancillary_da_rd', 'ancillary_da_rmd',
       'ancillary_da_rmu', 'ancillary_da_ru', 'ancillary_da_sr',
       'ancillary_rt_nr', 'ancillary_rt_rd', 'ancillary_rt_rmd',
       'ancillary_rt_rmu', 'ancillary_rt_ru', 'ancillary_rt_sr',
       'congestion_da', 'energy_da', 'unnamed: 0', 'gas_price', 'hour',
       'gen_geothermal', 'gen_biomass', 'gen_biogas', 'gen_small_hydro',
       'gen_wind', 'gen_solar_pv', 'gen_solar_thermal', 'gen_renewables',
       'gen_nuclear', 'gen_thermal', 'gen_imports', 'gen_hydro', 'load_2da',
       'load_7da', 'load_rt', 'load_da', 'load_rt.1', 'load_rtpd', 'losses_da',
       'ancillary_da_nr_mw', 'ancillary_da_rd_mw', 'ancillary_da_rmd_mw',
       'ancillary_da_rmu_mw', 'ancillary_da_ru_mw', 'ancillary_da_sr_mw',
       'ancillary_rt_nr_mw', 'ancillary_rt_rd_mw', 'ancillary_rt_rmd_mw',
       'ancillary_rt_rmu_mw', 'ancillary_rt_ru_mw', 'ancillary_rt_sr_mw',
       'congestion_rt', 'energy_rt', 'losses_rt', 'Unnamed: 0', 'Unn

In [622]:
new_data["CAISO"]["total"].to_csv("/Users/evanlipton/Dropbox (YSE)/Solar_ElectricData/Data/CAISO/all_data.csv")